# Aprendizaje de estructuras

> **Objetivos:**
> - Entender la función de verosimilitud máxima para el aprendizaje de estructuras en Redes Bayesianas.
> - Estudiar la función BIC para el aprendizaje de estructuras en Redes Bayesianas.

## 1. Introducción

### ¿Por qué aprender la estructura?

La **estructura** de una red bayesiana describe qué variables están conectadas entre sí y cómo fluye la información entre ellas. Cada arco representa una relación de dependencia, y su ausencia representa una independencia condicional fundamental para la factorización del modelo. En particular, una estructura correcta permite descomponer la distribución conjunta como:

$$
P(X_1, \dots, X_n) = \prod_{i=1}^n P(X_i \mid \text{Pa}(X_i)),
$$

donde $\text{Pa}(X_i)$ es el conjunto de padres del nodo $X_i$.


Aprender esta estructura es crucial cuando no contamos con un expert@ que la defina manualmente o cuando queremos descubrir relaciones que no son evidentes a simple vista. Una estructura bien aprendida evita dependencias falsas, reduce la complejidad del modelo, mejora la capacidad predictiva y ofrece interpretabilidad al revelar qué variables influyen realmente sobre otras. Por el contrario, una estructura incorrecta —al omitir arcos necesarios o incluir arcos espurios— distorsiona las independencias del dominio y afecta tanto la inferencia como la estimación de parámetros.

## 2. Métodos para aprender estructuras

Los métodos para aprender la estructura de una red bayesiana se dividen principalmente en dos enfoques: **constraint-based** y **score-based**. Ambos buscan determinar qué dependencias existen entre las variables, pero lo hacen desde perspectivas distintas.

### 2.1. Constraint-based

El enfoque **constraint-based** se basa en detectar independencias condicionales en los datos. La idea central es que, si dos variables resultan independientes dado un conjunto adecuado de condicionantes, entonces no debe existir un arco entre ellas. Estos métodos construyen primero un grafo no dirigido usando tests de independencia y luego orientan las aristas siguiendo reglas que garantizan la ausencia de ciclos. El algoritmo más representativo es el **PC Algorithm**, que es usado para dominios grandes y aprovecha la estructura de indepencias para limitar la búsqueda.

#### 2.1.1. Constraint-Based Learning (PC Algorithm)

Este método aprende la estructura detectando **independencias condicionales** en los datos.

La idea es simple: si dos variables son independientes dado un conjunto de variables, entonces **no deben tener un arco** en el DAG final.

El algoritmo PC:
1. Empieza con un grafo totalmente conectado.  
2. Elimina aristas si encuentra independencias condicionales.  
3. Orienta las aristas restantes evitando ciclos.  

Es adecuado cuando existen suficientes datos para hacer tests estadísticos confiables.

In [1]:
from pgmpy.utils import get_example_model
from pgmpy.sampling import BayesianModelSampling
from pgmpy.estimators import PC
import pandas as pd

# Cargar modelo un modelo de ejemplo
model_asia = get_example_model("asia")
sampler = BayesianModelSampling(model_asia)
data = sampler.forward_sample(size=5000)

# Aplicar PC Algorithm
pc = PC(data)
estimated_model = pc.estimate(return_type="dag")

estimated_model.edges()

  0%|          | 0/8 [00:00<?, ?it/s]

INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'asia': 'C', 'tub': 'C', 'smoke': 'C', 'lung': 'C', 'bronc': 'C', 'either': 'C', 'xray': 'C', 'dysp': 'C'}


  0%|          | 0/5 [00:00<?, ?it/s]

OutEdgeView([('bronc', 'dysp'), ('either', 'xray'), ('either', 'dysp'), ('tub', 'either'), ('lung', 'either'), ('smoke', 'bronc'), ('smoke', 'lung')])

### 2.2. Score-based

Por otro lado, los métodos **score-based** asignan una puntuación a cada estructura posible y seleccionan aquella que maximiza un criterio dado. Una estructura recibe un **score** que mide qué tan bien explica los datos, por ejemplo con el likelihood, el BIC o el score bayesiano BDeu. La búsqueda de la estructura puede hacerse mediante heurísticas como *Hill Climbing*, que exploran el vecindario de grafos posibles en busca de una mejora en la puntuación. Estos métodos permiten incorporar penalizaciones por complejidad y manejar ruido en los datos de forma natural.

### 2.2.1. Score-Based Learning (Hill Climbing + Score)

Aquí no usamos tests de independencia, sino una función de puntuación (*score*) que evalúa qué tan buena es una estructura.

Pasos:
1. Elegimos un score (likelihood, BIC, BDeu).  
2. Generamos una estructura inicial.  
3. Exploramos vecindarios (agregar, borrar, invertir arco).
4. Aceptamos movimientos que mejoran el score.

Hill Climbing es la heurística más usada: simple, rápida y efectiva para dominios pequeños y medianos.

In [ ]:
from pgmpy.utils import get_example_model
from pgmpy.sampling import BayesianModelSampling
from pgmpy.estimators import HillClimbSearch
import pandas as pd

# Cargar un modelo ejemplo y simular datos
model_alarm = get_example_model("alarm")
sampler = BayesianModelSampling(model_alarm)
data = sampler.forward_sample(size=10000)

# Estructura con Hill Climbing + BIC
hc = HillClimbSearch(data)
best_model = hc.estimate(scoring_method=(data))

best_model.edges()[:10]   # primeros 10 arcos

## 3. Score Bayesiano (BDeu)

El score BDeu incorpora un prior Dirichlet que suaviza las estimaciones.
Es útil cuando:
- La base de datos es pequeña  
- Se desea incorporar información previa  
- Se busca robustez ante conteos bajos  

Es equivalente para estructuras en la misma clase de equivalencia.
